# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'Data/'
import pandas as pd
import numpy as np
import glob
import matplotlib
%pylab inline

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Solution

### Explanation
By looking at the data files from those 3 countries, we can see that there 3 types of new cases (confirmed, probable and suspected) that are represented every time by 3 different fields, and 1 field that corresponds to the number of new deaths. For those 3 countries, the data are represented in slightly different ways, using different terminologies. It thus makes sense to go over those 3 countries one after the other, clean, filter and uniformize the data, compute what we need (which is the average number of new cases and new deaths per day for each month) and then put those 3 countries and what we've computed together in a final DataFrame.

### Corresponding code

In [ ]:
# Constants
EBOLA_FOLDER = DATA_FOLDER + 'ebola/'
CONFIRMED_NEW_CASES = 'New confirmed cases'
PROBABLE_NEW_CASES = 'New probable cases'
SUSPECTED_NEW_CASES = 'New suspected cases'
NEW_DEATHS = 'New deaths'
AVG_CONFIRMED_NEW_CASES = 'New confirmed cases (avg per day)'
AVG_PROBABLE_NEW_CASES = 'New probable cases (avg per day)'
AVG_SUSPECTED_NEW_CASES = 'New suspected cases (avg per day)'
AVG_NEW_DEATHS = 'New deaths (avg per day)'

### DataFrames creation
For each country, we first have to read every CSV file and put their content into one big DataFrame. This will allow us to then work on the DataFrame (filter data, clean data, aggregate what we need, ...).

In [ ]:
file_gui = glob.glob(EBOLA_FOLDER + "guinea_data/*.csv")
file_lib = glob.glob(EBOLA_FOLDER + "liberia_data/*.csv")
file_sie = glob.glob(EBOLA_FOLDER + "sl_data/*.csv")

list_gui = []
# Create DataFrame for Guinea
for file in file_gui:
    df = pd.read_csv(file,header=0,index_col=None,parse_dates=['Date'])
    list_gui.append(df)
ebola_gui = pd.concat(list_gui)

list_lib = []
# Create DataFrame for Liberia
for file in file_lib:
    df = pd.read_csv(file,header=0,index_col=None,parse_dates=['Date'])
    list_lib.append(df)
ebola_lib = pd.concat(list_lib)

list_sie = []
# Create DataFrame for Sierra Leone
for file in file_sie:
    df = pd.read_csv(file,header=0,index_col=None,parse_dates=['date'])
    list_sie.append(df)
ebola_sie = pd.concat(list_sie)

### Guinea
Here we want to extract the average per day for every month for all 3 types of new cases and for new deaths in Guinea. What we have to do is to filter the data for each case (we have 4 cases) we want to have, get each time the corresponding number of new cases or deaths, and merge those 4 DataFrames into 1 by the date. Once this is done, we can extract the month and year from the date, group by the month and year, and compute the average as an aggregation.

In [ ]:
def get_guinea_specific_df(description, new_col_name):
    res = ebola_gui.where(lambda row: row['Description'] == description)
    df = res[pd.notnull(res['Description'])]
    df = df[['Totals', 'Date']]
    df = df.rename(columns = {'Totals': new_col_name})
    return df
    
new_cases_confirmed = get_guinea_specific_df('New cases of confirmed', CONFIRMED_NEW_CASES)
new_cases_probable = get_guinea_specific_df('New cases of probables', PROBABLE_NEW_CASES)
new_cases_suspect = get_guinea_specific_df('New cases of suspects', SUSPECTED_NEW_CASES)
# Note: This makes the first file (2014-08-04) disappear because it is registered under "New deaths registered today"
# and not "New deaths registered" like the others.
new_deaths_registered = get_guinea_specific_df('New deaths registered', NEW_DEATHS)

# Put all things together
guinea_df = new_cases_confirmed.merge(new_cases_probable, on='Date').merge(new_cases_suspect, on='Date').merge(new_deaths_registered, on='Date')
guinea_df['Country'] = 'Guinea'
#guinea_df = guinea_df.rename(columns={'Date': 'Month'})
guinea_df['Date'] = guinea_df.Date.dt.strftime("%B %Y")
guinea_df = guinea_df.apply(pd.to_numeric, errors='ignore')

# Group by to compute the average per day for each month
guinea_df = guinea_df.groupby(['Date', 'Country']).mean()
guinea_df

### Liberia
The idea here is similar. The big difference is that there are numbers that seem wrongly reported in December (you can see huge numbers, bigger than 1000), so we want to filter them out so that they don't mess up our data. To do that we choose a threshold (500) and if we find a value bigger than the threshold, we remove the row because it is very likely that this is a wrongly reported value.

In [ ]:
def get_liberia_specific_df(variable, new_col_name):
    res = ebola_lib.where(lambda row: row['Variable'] == variable)
    df = res[pd.notnull(res['Variable'])]
    df = df[pd.isnull(df['National']) | (df['National'] < 500.0)] # Remove wrongly reported data, but keep NaN for now
    df = df[['National', 'Date']]
    df = df.rename(columns = {'National': new_col_name})
    return df

new_cases_confirmed = get_liberia_specific_df('New case/s (confirmed)', CONFIRMED_NEW_CASES)
new_cases_probable = get_liberia_specific_df('New Case/s (Probable)', PROBABLE_NEW_CASES)
new_cases_suspect = get_liberia_specific_df('New Case/s (Suspected)', SUSPECTED_NEW_CASES)
new_deaths_registered = get_liberia_specific_df('Newly reported deaths', NEW_DEATHS)

# Put all things together
liberia_df = new_cases_confirmed.merge(new_cases_probable, on='Date').merge(new_cases_suspect, on='Date').merge(new_deaths_registered, on='Date')
liberia_df['Country'] = 'Liberia'
#liberia_df = liberia_df.rename(columns={'Date': 'Month'})
liberia_df['Date'] = liberia_df.Date.dt.strftime("%B %Y")
liberia_df = liberia_df.apply(pd.to_numeric, errors='ignore')

# Group by to compute the average per day for each month
liberia_df = liberia_df.groupby(['Date', 'Country']).mean()
liberia_df

### Sierra Leone
Again, the idea is the same. But this time, we don't have a number that gives us exactly the new number of deaths. So instead, we are using 'etc_new_deaths' for computing the number of deaths per day; 'etc' means 'Ebola Treatment Center', so those seem to be the deaths happening in the treatment center.
Another solution would have been to use 'death_confirmed', 'death_suspected' and 'death_probable', but those values are cumulative, so we would have had to compute the differences every day, and be sure that we have data every day to have accurate values, but this is not the case here.

In [ ]:
def get_sl_specific_df(variable, new_col_name):
    res = ebola_sie.where(lambda row: row['variable'] == variable)
    df = res[pd.notnull(res['variable'])]
    df = df[['National', 'date']]
    df = df.rename(columns = {'National': new_col_name})
    return df

new_cases_confirmed = get_sl_specific_df('new_confirmed', CONFIRMED_NEW_CASES)
new_cases_probable = get_sl_specific_df('new_probable', PROBABLE_NEW_CASES)
new_cases_suspect = get_sl_specific_df('new_suspected', SUSPECTED_NEW_CASES)
new_deaths_registered = get_sl_specific_df('etc_new_deaths', NEW_DEATHS)

# Put all things together
sl_df = new_cases_confirmed.merge(new_cases_probable, on='date').merge(new_cases_suspect, on='date').merge(new_deaths_registered, on='date')
sl_df['Country'] = 'Sierra Leone'
#total = total.rename(columns={'date': 'Month'})
sl_df['date'] = sl_df.date.dt.strftime("%B %Y")
sl_df = sl_df.apply(pd.to_numeric, errors='ignore')

# Group by to compute the average per day for each month
sl_df = sl_df.groupby(['date', 'Country']).mean()
sl_df

### Put everything together
Finally, we just want to concatenate everything together and rename the different columns accordingly.

In [ ]:
# Concatenate the values for the 3 countries and rename columns accordingly
final_df = pd.concat([guinea_df, liberia_df, sl_df])
final_df = final_df.rename(columns={
    CONFIRMED_NEW_CASES: AVG_CONFIRMED_NEW_CASES,
    PROBABLE_NEW_CASES: AVG_PROBABLE_NEW_CASES,
    SUSPECTED_NEW_CASES: AVG_SUSPECTED_NEW_CASES,
    NEW_DEATHS: AVG_NEW_DEATHS})
final_df = final_df.fillna('unknown')
final_df

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

## Solution

### Explanation

Here we have 9 spreedsheets containing microbiome data. This data can be seen as a single table containing 2 columns, the `NAME` and the `COUNT`. Thus, we simply read those files and concatenate them into a single dataframe representing our table with 2 columns.

Given that the file does not contain headers, we ignore the first row (which is not a header) and give a name to the columns with the `header=None` and `names=['NAME', 'COUNT']` parameters.

We also add a new `BARCODE` column to the dataframe, which is the name corresponding to the column name in the metadata file. We give it the same name in order to have normalized dataframes that can be merged on columns with the same name.

Finally, we read the metadata file into a dataframe that we merge with our initial dataframe, we replace NA values with the `unknown` value and place an index on `[BARCODE, NAME]` in order to have unique indexes

### Corresponding Code

In [ ]:
MICROBIOME_FOLDER = DATA_FOLDER + 'microbiome/'
mid_files = glob.glob(MICROBIOME_FOLDER + "MID*.xls")

In [ ]:
res = ebola_gui.where(ebola_gui['Description'] == 'New cases of confirmed')
df = res[pd.notnull(res['Description'])]
new_cases_confirmed = df[['Totals', 'Date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'Totals': 'New cases'})

res = ebola_gui.where(ebola_gui['Description'] == 'New deaths registered')
df = res[pd.notnull(res['Description'])]
new_deaths_registered = df[['Totals', 'Date']]
new_deaths_registered = new_deaths_registered.rename(columns = {'Totals': 'New deaths'})
test = new_cases_confirmed.merge(new_deaths_registered, on='Date')
test['Country'] = 'Guinea'
test = test.rename(columns={'Date': 'Month'})
test['Month'] = test.Month.dt.strftime("%B")
test

In [ ]:
res = ebola_lib.where(ebola_lib['Variable'] == 'New case/s (confirmed)')
df = res[pd.notnull(res['Variable'])]
new_cases_confirmed = df[['National', 'Date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'National': 'New cases'})
new_cases_confirmed

In [ ]:
res = ebola_lib.where(ebola_lib['Variable'] == 'New case/s (confirmed)')
df = res[pd.notnull(res['Variable'])]
new_cases_confirmed = df[['National', 'Date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'National': 'New cases'})

res = ebola_lib.where(ebola_lib['Variable'] == 'Newly reported deaths')
df = res[pd.notnull(res['Variable'])]
new_deaths_registered = df[['National', 'Date']]
new_deaths_registered = new_deaths_registered.rename(columns = {'National': 'New deaths'})
test = new_cases_confirmed.merge(new_deaths_registered, on='Date')
test['Country'] = 'Liberia'
test = test.rename(columns={'Date': 'Month'})
test['Month'] = test.Month.dt.strftime("%B")
test

In [ ]:
ebola_lib[list(['Date','Variable','National'])]

In [ ]:
# We are using 'etc_new_deaths' for computing the number of deaths per day because it's the only information we have.
# 'etc' means 'Ebola Treatment Center', so those seem to be the deaths happening in the treatment center.

res = ebola_sie.where(ebola_sie['variable'] == 'new_confirmed')
df = res[pd.notnull(res['variable'])]
new_cases_confirmed = df[['National', 'date']]
new_cases_confirmed = new_cases_confirmed.rename(columns = {'National': 'New cases'})
#mean = np.ceil(np.mean(list(map(int, new_cases_confirmed['New cases'].dropna().values))))
#new_cases_confirmed = new_cases_confirmed.fillna(mean)

res = ebola_sie.where(ebola_sie['variable'] == 'etc_new_deaths')
df = res[pd.notnull(res['variable'])]
new_deaths_registered = df[['National', 'date']]
new_deaths_registered = new_deaths_registered.rename(columns = {'National': 'New deaths'})
#mean = np.ceil(np.mean(list(map(int, new_deaths_registered['New deaths'].dropna().values))))
#new_deaths_registered = new_deaths_registered.fillna(mean)
total = new_cases_confirmed.merge(new_deaths_registered, on='date')
total['Country'] = 'Sierra Leone'
total = total.rename(columns={'date': 'Month'})
total['Month'] = total.Month.dt.strftime("%B")
total

In [ ]:
ebola_sie[list(['date','variable','National'])]

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

## Solution

### Explanation

Here we have 9 spreedsheets containing microbiome data. This data can be seen as a single table containing 2 columns, the `NAME` and the `COUNT`. Thus, we simply read those files and concatenate them into a single dataframe representing our table with 2 columns.

Given that the file does not contain headers, we ignore the first row (which is not a header) and give a name to the columns with the `header=None` and `names=['NAME', 'COUNT']` parameters.

We also add a new `BARCODE` column to the dataframe, which is the name corresponding to the column name in the metadata file. We give it the same name in order to have normalized dataframes that can be merged on columns with the same name.

Finally, we read the metadata file into a dataframe that we merge with our initial dataframe, we replace NA values with the `unknown` value and place an index on `[BARCODE, NAME]` in order to have unique indexes

### Corresponding Code

In [ ]:
MICROBIOME_FOLDER = DATA_FOLDER + 'microbiome/'
mid_files = glob.glob(MICROBIOME_FOLDER + "MID*.xls")

In [ ]:
# Initial empty dataframe
mid = pd.DataFrame()
for idx, file in enumerate(mid_files, 1):
    file_name = 'MID' + str(idx)
    # We give appropriate names to colummns and replace NA values with "unknown"
    tmp_mid = pd.read_excel(MICROBIOME_FOLDER+file_name+'.xls', names=['NAME', 'COUNT'], header=None)
    # We add a column "BARCODE" corresponding to the name of the file being read
    tmp_mid['BARCODE'] = file_name
    # We concatenate the values to create a single dataframe
    mid = pd.concat([mid, tmp_mid])

# We read the metadata file and merge it to the MID dataframe where columns have the same name
metadata = pd.read_excel(MICROBIOME_FOLDER + 'metadata.xls')
# We create an unique index on BARCODE and TAXON and replace NA values by 'unknown'
final_df = pd.merge(mid, metadata).fillna('unknown').set_index(['BARCODE', 'NAME'])
final_df

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
titanic = pd.read_excel('Data/titanic.xls')
titanic

1) In the follow output we can see the different type of attributes with their unique values

In [ ]:
for c in titanic.columns:
    print("%s is of type %s" % (c,titanic[c].dtype))
    print(titanic[c].unique())
    print("\n ------------------------------\n")

From above output we can see that travel class, surviving, sex and embarked have some limited possibilities,
which make them good candidate for categorical type

In [ ]:
#change travel class to categorical type with more significant label
titanic['pclass'] = titanic['pclass'].apply(lambda x: {1:'1st',2:'2nd',3:'3rd'}[x]).astype('category')
#change surviving attribute to categorical type
titanic['survived'] = titanic['survived'].astype('category')
#change sex attribute to categorical type
titanic['sex'] = titanic['sex'].astype('category')
#change embarked attribute to categorical type with more significant label
titanic['embarked'] = titanic['embarked'].apply(lambda x : {'C':'Cherbourg','Q':'Queenstown','S':'Southampton'}.get(x,'Unkown'))
titanic.pclass.dtype

2) Histogram plot for travel class, embarkation port, sex and age by decade

In [ ]:
hist_pclass = titanic['pclass']
hist_pclass.value_counts().plot.bar()

In [ ]:
hist_emb = titanic['embarked'].value_counts().plot.bar()

In [ ]:
hist_sex = titanic['sex'].value_counts().plot.bar()

In [ ]:
hist_age = titanic['age'].plot.hist(bins=10)

3) Proportion of passengers by cabin floor

In [ ]:
cabin_plot = titanic['cabin'].astype(str).apply(lambda x : x[0]).apply(lambda x : {'n':'unkown'}.get(x,x)).copy()
cabin_plot.value_counts().plot.pie(autopct='%.2f',figsize=(8, 8))

4) Proportion of the passengers that survived for each travel class

In [ ]:
#filter needed columns
survived_data = titanic[['pclass','survived']].copy()
#created columns for people who died
survived_data['died'] = 1 - survived_data['survived'].astype('int')
#convert survived column to int type
survived_data['survived'] = survived_data['survived'].astype('int')
#group by travel class on a sum and transpose (for ploting to works)
survived_plot = survived_data.groupby('pclass').sum().transpose()
#subplot data on 3 pie charts
survived_plot.plot.pie(subplots=True,autopct='%.2f',figsize=(12, 4))

5) Proportion of the passengers that survived by travel class and sex

In [ ]:
#filter needed columns
survived_sex_data = titanic[['pclass','sex','survived']].copy()
#creating new columns for each class of pair travel class and sex
survived_sex_data['class'] =  survived_sex_data['pclass'].astype(str) + " " + survived_sex_data['sex'].astype(str)
#filter out columns we don't need anymore
sur_sex_plot = survived_sex_data[['survived','class']].copy()
#convert survived categorical type to int
sur_sex_plot['survived'] = sur_sex_plot['survived'].astype('int')
#group by class of pairs and plot
sur_sex_plot.groupby('class').sum().sort_values(['survived']).plot.bar()

6) Survival proportions by age category, travel class and sex

In [ ]:
#filter needed columns
prop_data = titanic[['pclass','survived','age','sex']].copy().sort_values(['age'])
prop_data
#compute median
median = np.ceil(len(prop_data['age'])/2)
median_val = prop_data['age'][median]
#create new age category
prop_data['age_cat'] = prop_data['age'].apply(lambda x : 'young' if x < median_val else 'old')
#create new class with age category, travel class and sex
prop_data['class'] = prop_data['age_cat'].astype(str) + " " + prop_data['pclass'].astype(str) + " " + prop_data['sex'].astype(str)
#filter out non needed columns
prop_df = prop_data[['survived','class']].copy()
#converting categorical type to int
prop_df['survived'] = prop_df['survived'].astype('int')
#group by new class
prop_df.groupby(['class']).sum().sort_values(['survived'],ascending=[False])